<h1>Organizar la base de datos.</h1>

In [73]:
import pandas as pd
import numpy as np
db = pd.read_csv("BD.txt", delimiter="\t")
print(db)
y = np.zeros(100)
#Archivo solo con los documentos
f = open ('Docs.txt','w')
for i in range(len(db)):
    y[i] = db['Clase'][i]
    f.write(db['Documentos'][i] + '\n')
f.close()

                                           Documentos  Clase
0   Excelente. Recomendable. Habitaciones cómodas ...      1
1   Las habitaciones muy amplias así como la cama....      1
2   La piscina y el bungalow son muy espaciosos y ...      1
3   El desayuno bufett bastante bueno, cuenta con ...      1
4   La ubicación del hotel es perfecta para descan...      1
5   Muy bien el transporte de y hacia el aeropuert...      1
6   Las instalaciones en general estupendas, nuest...      1
7   La piscina fantástica. Perfecto para un respir...      1
8   La piscina principalmente. Tienes servicio de ...      1
9   El sitio es excelente y la atencion un lujo.. ...      1
10  La habitación era un maravilla, amplia y confo...      1
11  Muy lindo, ubicado en zona residencial muy linda.      1
12  La habitación muy limpia, excelente el conduct...      1
13  Buen trato y la tranquilidad. A parte de estar...      1
14  Todo el hotel está muy bueno  sin problemas es...      1
15  Un hotel perfecto pa

In [74]:
print(y)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


# Importar la base de datos de MLSenticon

In [75]:
#-*- coding: utf-8 -*-
import requests

sentimientos = open('MLSenticon.txt', 'r')

MLSenticonDic = { }
for word in sentimientos:
    palabra = word.split('\t')[0]
    valor = float(word.split('\t')[1][:-1])
    MLSenticonDic[palabra] = valor

## Caracterización para Positivos

In [76]:
def CalcularPolaridad(sentence):
    for word in sentence:
        palabra =  str(word['token'])
        if palabra.lower() == 'no' or palabra.lower() == 'pero':
            return -1 #Cambia la polaridad
    return 1 #No cambia la polaridad

In [77]:
Caracteristicas = []
for i in range(50):
    vectorCaracteristicasPos = [0,0]
    
    f = open ('Coment.txt','w')
    f.write(db['Documentos'][i] + '\n')
    f.close()
    print(db['Documentos'][i])
    
    #Archivo a ser enviado
    files = {'file': open('Coment.txt', 'r')}
    
    #Parámetros
    params = {'outf': 'tagged', 'format': 'json'}

    #Enviar petición
    url = "http://www.corpus.unam.mx/servicio-freeling/analyze.php"
    r = requests.post(url, files=files, params=params)
    #print (r)

    #Convertir de formato json
    obj = r.json()

    #pint(len(obj))    #obj es una lista de listas. Tiene tantos elementos (listas) como frases tenga el file
    
    #Ejemplo, obtener todos los lemas
    for sentence in obj:
        #print (sentence)    #sentence es una lista de diccionarios. Tiente tantos elementos (diccionarios)
                             #como palabras tiene la frase
        polaridad = CalcularPolaridad(sentence)
        for word in sentence:
            #word es un diccionario con 4 claves: token, lemma, tag y prob(probabilidad de que el tag haya sido bien asignado)
            #A través de esas cuatro claves se podrá acceder a la información que requerimos para
            #Comenzar a construir el vector de características de un texto
            print(word)
            if (word['tag'][0] == 'A'):
                    lema = word['lemma']
                    print(lema)
                    
                    try: 
                        valor = MLSenticonDic[lema]
                    except:
                        valor = 0

                    if polaridad == 1:
                        if (valor > 0):
                            vectorCaracteristicasPos[0] += valor
                        else:
                            vectorCaracteristicasPos[1] += valor
                    else:
                        if (valor > 0):
                            vectorCaracteristicasPos[1] += (valor * polaridad)
                        else:
                            vectorCaracteristicasPos[0] += (valor * polaridad)

        #print(vectorCaracteristicasPos)
    Caracteristicas.append(vectorCaracteristicasPos)
#print(Caracteristicas)

Excelente. Recomendable. Habitaciones cómodas y restaurante con buena comida. Cerca al aeropuerto.


ConnectionError: HTTPConnectionPool(host='www.corpus.unam.mx', port=80): Max retries exceeded with url: /servicio-freeling/analyze.php?outf=tagged&format=json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001ED1A3B5F60>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

## Caracterización para negativos

In [72]:
for i in range(50,100):
    vectorCaracteristicasNeg = [0,0]
    
    f = open ('Coment.txt','w')
    f.write(db['Documentos'][i] + '\n')
    f.close()
    #print(db['Documentos'][i])
    
    #Archivo a ser enviado
    files = {'file': open('Coment.txt', 'r')}
    
    #Parámetros
    params = {'outf': 'tagged', 'format': 'json'}

    #Enviar petición
    url = "http://www.corpus.unam.mx/servicio-freeling/analyze.php"
    r = requests.post(url, files=files, params=params)
    #print (r)

    #Convertir de formato json
    obj = r.json()

    #pint(len(obj))    #obj es una lista de listas. Tiene tantos elementos (listas) como frases tenga el file
    
    #Ejemplo, obtener todos los lemas
    for sentence in obj:
        #print (sentence)    #sentence es una lista de diccionarios. Tiente tantos elementos (diccionarios)
                             #como palabras tiene la frase
        polaridad = CalcularPolaridad(sentence)
        for word in sentence:
            #word es un diccionario con 4 claves: token, lemma, tag y prob(probabilidad de que el tag haya sido bien asignado)
            #A través de esas cuatro claves se podrá acceder a la información que requerimos para
            #Comenzar a construir el vector de características de un texto
            #print(word)
            if (word['tag'][0] == 'A'):
                    lema = word['lemma']
                    #print(lema)

                    try: 
                        valor = MLSenticonDic[lema]
                    except:
                        valor = 0
                        
                    if polaridad == 1:
                        if (valor > 0):
                            vectorCaracteristicasNeg[0] += valor
                        else:
                            vectorCaracteristicasNeg[1] += valor
                    else:
                        if (valor > 0):
                            vectorCaracteristicasNeg[1] += (valor * polaridad)
                        else:
                            vectorCaracteristicasNeg[0] += (valor * polaridad)
                    

        #print(vectorCaracteristicasNeg)
    Caracteristicas.append(vectorCaracteristicasNeg)
#print(Caracteristicas)

TypeError: '_io.TextIOWrapper' object is not subscriptable

<h1>Regresión Logistica</h1>

In [45]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier as KNN

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier as RF

def error_measures(Yestimado, Yteorico):
    
    CM = confusion_matrix(Yteorico, Yestimado)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    
    sens = TP/(TP+FN)
    esp = TN/(TN+FP)
    
    return sens, esp



Muestras training:  70 %
Muestras testing:  30 %

Resultados con Regresión logística (Lineal)

Accuracy:  0.8643333333333335 +/- 0.05275835900067821
Sensitivity:  0.8611223116408876 +/- 0.08124416280084633
Especificity:  0.8685351238458823 +/- 0.08231733204795762


In [51]:
def RegresionLogistica(db):
    lr=LogisticRegression()
    acc = []
    sens = []
    esp = []

    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(db,y,test_size=0.3)   #Preguntar bootstraping


        lr.fit(Xtrain,Ytrain)
        Yest = lr.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(lr.score(Xtest,Ytest))

    print("Muestras training: ", round((np.size(Xtrain,0)*100)/100), "%")
    print("Muestras testing: ", round((np.size(Xtest,0)*100)/100), "%")

    print("\nResultados con Regresión logística (Lineal)\n")
    print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
    print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
    print("Especificity: ", np.mean(esp), "+/-", np.std(esp))
    
RegresionLogistica(Caracteristicas)

Muestras training:  70 %
Muestras testing:  30 %

Resultados con Regresión logística (Lineal)

Accuracy:  0.8656666666666668 +/- 0.05526803375872489
Sensitivity:  0.8681680518483923 +/- 0.08644053034092267
Especificity:  0.8680835774562554 +/- 0.0785921866641888


## K-nearest neighbors

In [52]:
def KNearestNeighbors(bd):
    acc = []
    sens = []
    esp = []

    #Complete el código para KNN
    neigh = KNN(n_neighbors=40)

    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bd,y,test_size=0.3)   

        neigh.fit(Xtrain,Ytrain)
        Yest = neigh.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(neigh.score(Xtest,Ytest))


    print("\nResultados con K vecinos más cercanos\n")
    print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
    print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
    print("Especificity: ", np.mean(esp), "+/-", np.std(esp))
    
KNearestNeighbors(Caracteristicas)


Resultados con K vecinos más cercanos

Accuracy:  0.813 +/- 0.06581033353509161
Sensitivity:  0.7840013414518059 +/- 0.10863710637777702
Especificity:  0.8541898397010627 +/- 0.09250932409638464


## Random Forest

In [53]:
def RandomForest(bd):
    acc = []
    sens = []
    esp = []
    clf = RF(n_estimators=400)

    #Complete el código para Árboles de decisión
    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bd,y,test_size=0.3)  

        clf.fit(Xtrain,Ytrain)
        Yest = clf.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(clf.score(Xtest,Ytest))


    print("Resultados con Random Forest\n")
    print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
    print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
    print("Especificity: ", np.mean(esp), "+/-", np.std(esp))

RandomForest(Caracteristicas)

Resultados con Random Forest

Accuracy:  0.8646666666666667 +/- 0.05329582014046172
Sensitivity:  0.8228421946990058 +/- 0.09490833880144853
Especificity:  0.907980195374091 +/- 0.078953943744423


## Vectorización 

In [63]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Cargando los datos desde el archivo DB.txt (Noticias de periódico)
db = open('Docs.txt', 'r')
db_data = []
for i in db:
    db_data.append(i)
db.close()

#print(db_target)  #Las clases son 1->textos positivos; 0->tetos negativos


Estaba muy sucio. Creo que debería recibir una inspección de sanidad. No es conveniente pisar descalzo en la habitación, la moqueta está asquerosa. La entrada a recepción estaba sucia y el suelo pegajoso. Por otra parte llegamos cerca de las 12 de la noche y había muchos huéspedes extranjeros que no paraban de dar voces.



In [69]:
vector = CountVectorizer(ngram_range=(2,2))    #Matriz de términos en documentos

vector.fit(db_data)

bow = vector.transform(db_data)

RegresionLogistica(bow)
KNearestNeighbors(bow)
RandomForest(bow)

Muestras training:  70 %
Muestras testing:  30 %

Resultados con Regresión logística (Lineal)

Accuracy:  0.7369999999999999 +/- 0.08177136146326267
Sensitivity:  0.9145263623545358 +/- 0.07640277432365478
Especificity:  0.5755475179705057 +/- 0.15507702957336375

Resultados con K vecinos más cercanos

Accuracy:  0.49566666666666664 +/- 0.06890492644861387
Sensitivity:  1.0 +/- 0.0
Especificity:  0.0 +/- 0.0
Resultados con Random Forest

Accuracy:  0.612 +/- 0.1113073822050152
Sensitivity:  0.8925737874097007 +/- 0.21337200270576043
Especificity:  0.3977519072344925 +/- 0.2756208306166685


In [68]:
vector = TfidfVectorizer()   #Esquema TF-IDF

vector.fit(db_data)

bow = vector.transform(db_data)

RegresionLogistica(bow)
KNearestNeighbors(bow)
RandomForest(bow)

Muestras training:  70 %
Muestras testing:  30 %

Resultados con Regresión logística (Lineal)

Accuracy:  0.8913333333333334 +/- 0.07536872325072545
Sensitivity:  0.8841014451664606 +/- 0.12324883747451106
Especificity:  0.9216038476001794 +/- 0.09445383219748275

Resultados con K vecinos más cercanos

Accuracy:  0.7556666666666666 +/- 0.08832452786300206
Sensitivity:  0.5810402467759278 +/- 0.15535875614401923
Especificity:  0.9539374104567603 +/- 0.05654930881667824
Resultados con Random Forest

Accuracy:  0.8486666666666665 +/- 0.0772326931350247
Sensitivity:  0.9565039838472501 +/- 0.05258003831807601
Especificity:  0.7624861066436143 +/- 0.14041336496483248
